In [1]:
import configparser
import requests
import time
import sys

from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5 import uic

In [2]:
class Access:
    
    def __init__(self):
        self.url = 'https://api.korbit.co.kr/v1/oauth2/access_token'
        self.config = configparser.ConfigParser()
        self.config.read('C:\\Users\\김민수\\Desktop\\korbit.ini')
        self.token = None
        self.token_refresh = None
        
    # 최초 토큰 요청
    def request_token(self):
        data = {
            'client_id' : self.config['KORBIT']['client_id'],
        'client_secret' : self.config['KORBIT']['client_secret'],
        'username' : self.config['KORBIT']['username'],
        'password' : self.config['KORBIT']['password'],
        'grant_type' : 'password'}

        res = requests.post(self.url, data=data)
        res = res.json()
        token = res['access_token']
        token_refresh = res['refresh_token']
        self.token = token
        self.token_refresh = token_refresh
        print(self.token)
        return token, token_refresh
        
    # 토큰 갱신
    def refresh(self):
        data = {
            'client_id' : self.config['KORBIT']['client_id'],
        'client_secret' : self.config['KORBIT']['client_secret'],
        'refresh_token' : self.token_refresh,
        'grant_type' : 'refresh_token'}

        res = requests.post(self.url, data=data)
        res = res.json()
        return res
    
    # 사용자 정보
    def user(self):
        url = 'https://api.korbit.co.kr/v1/user/info'
        headers = {
        "Authorization" : "Bearer " + self.token}
        
        response = requests.get(url, headers=headers)
        res = response.json()
        return res

    # 매수 (지정가)
    def buy(self):
        headers = {
        'Authorization': 'Bearer ' + self.token
        }

        data = {
          'currency_pair': 'etc_krw',
          'type': 'limit',
          'price' : '100000',
          'coin_amount': '1',
          'nonce': str(int(time.time()))
        }

        response = requests.post('https://api.korbit.co.kr/v1/user/orders/buy', headers=headers, data=data)
        res = response.json()
        return res
    
    # 매도 (시장가)
    def sell(self):
        headers = {
        'Authorization': 'Bearer ' + self.token
        }

        data = {
          'currency_pair': 'etc_krw',
          'type': 'limit',
          'price' : '100000',
          'coin_amount': '1',
          'nonce': str(int(time.time()))
        }

        response = requests.post('https://api.korbit.co.kr/v1/user/orders/sell', headers=headers, data=data)
        res = response.json()
        return res
    
    # 주문조회
    def look(self):
        headers = {
        'Authorization': 'Bearer ' + self.token
        }

        response = requests.get('https://api.korbit.co.kr/v1/user/orders', headers=headers)
        res = response.json()
        return res
    
    def look_inout(self):
        headers = {
        'Authorization': 'Bearer ' + self.token
        }

        response = requests.get('https://api.korbit.co.kr/v1/user/transfers', headers=headers)
        res = response.json()
        return res
    
    # 지갑잔고 조회
    def bal(self):
        headers = {
        'Authorization': 'Bearer ' + self.token,
        }

        response = requests.get('https://api.korbit.co.kr/v1/user/balances', headers=headers)
        res = response.json()
        return res
    
    # 주소할당
    def assign(self):
        headers = {
        'Authorization': 'Bearer ' + self.token,
        }

        data = {
          'currency': 'btc',
          'nonce': str(int(time.time()))
        }

        response = requests.post('https://api.korbit.co.kr/v1/user/coins/address/assign', headers=headers, data=data)
        res = response.json()
        return res
    
    # 지갑 조회
    def look_bal(self):
        headers = {'Authorization' : 'Bearer ' + self.token}
        response = requests.get('https://api.korbit.co.kr/v1/user/accounts', headers=headers)
        res = response.json()
        return res

In [3]:
# public api
class Public_api:
    def __init__(self):
        pass
    
    # 최종 체결가격
    def final(self):
        params = {'currency_pair' : 'btc_krw'}
        response = requests.get('https://api.korbit.co.kr/v1/ticker', params=params)
        res = response.json()
        return res
    
    # 시장 상세현황
    def detail(self):
        params = {'currency_pair', 'btc_krw'}
        response = requests.get('https://api.korbit.co.kr/v1/ticker/detailed', params=params)
        res = response.json()
        return res
    
    # 모든 시장 상세현황
    def detail_all(self):
        response = requests.get("https://api.korbit.co.kr/v1/ticker/detailed/all")
        res = response.json()
        return res
    
    # 매수/매도 호가
    def orderbook(self):
        params = {'currency_pair' : 'btc_krw'}
        response = requests.get('https://api.korbit.co.kr/v1/orderbook', params=params)
        res = response.json()
        return res

    # 체결내역
    def contract(self):
        params = {'currency_pair' : 'btc_krw'}
        response = requests.get('https://api.korbit.co.kr/v1/transactions', params=params)
        res = response.json()
        return res
    
    # 제약조건
    def regulate(self):
        url = 'https://api.korbit.co.kr/v1/constants'
        response = requests.get(url)
        res = response.json()
        return res

In [141]:
papi = Public_api()
papi.final()

{'timestamp': 1564999233125, 'last': '11640000'}


In [6]:
form = uic.loadUiType('C:\\Users\\김민수\\Desktop\\trading_tmp\\korbit_trader.ui')[0] # UI 로드

class myWindow(QMainWindow, form):
    def __init__(self):
        super().__init__()
        self.setupUi(self) # UI 셋팅
        self.pub_api = Public_api()
        self.priv_api = Access()
        self.access()
        self.timer()
        
    def access(self):
        self.token, self.token_refresh = self.priv_api.request_token()
    
    def timer(self):
        self.timer_account = QTimer()
        self.timer_account.timeout.connect(self.update) # 잔고 조회 함수 연결
        self.timer_account.setSingleShot(False)
        self.timer_account.start(3000)
        
    def update(self):
        bal = self.priv_api.bal()
        #cur = self.pub_api.detail()
        print(bal)
        self.label_remain_krw = bal
        self.label_remain_btc = bal
        #self.label_cur_btc = cur
        #self.label_cur_eth = cur
        #self.label_cur_xrp = cur

In [7]:
if __name__ == "__main__":
    
    # 창 닫아도 주피터에서 다시 열수있게 - cmd 실행땐 삭제
    app = QApplication.instance()
    if app is None:
        app = QApplication(sys.argv)
    else:
        print('QApplication instance already exists: %s' % str(app))

    window = myWindow()
    window.show()

    app.exec_()
    app.exit()

QApplication instance already exists: <PyQt5.QtWidgets.QApplication object at 0x000001BF0E62F9D8>
OfaSD4NH3FqYUMtdNgbtYxU3RdPNBAeR294rJrSoU0jJs908MxpAHEzZuatSm
{'aergo': {'available': '0', 'trade_in_use': '0', 'withdrawal_in_use': '0'}, 'bat': {'available': '0', 'trade_in_use': '0', 'withdrawal_in_use': '0'}, 'bch': {'available': '0', 'trade_in_use': '0', 'withdrawal_in_use': '0', 'avg_price': '3320000', 'avg_price_updated_at': 1527619997000}, 'bsv': {'available': '0', 'trade_in_use': '0', 'withdrawal_in_use': '0', 'avg_price': '90000', 'avg_price_updated_at': 1542353520000}, 'btc': {'available': '0', 'trade_in_use': '0', 'withdrawal_in_use': '0', 'avg_price': '7478237.917054124751869471', 'avg_price_updated_at': 1533792780070}, 'btg': {'available': '0', 'trade_in_use': '0', 'withdrawal_in_use': '0', 'avg_price': '427500', 'avg_price_updated_at': 1527620005000}, 'dash': {'available': '0', 'trade_in_use': '0', 'withdrawal_in_use': '0'}, 'etc': {'available': '0', 'trade_in_use': '0', 'wi

In [2]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry


def requests_retry_session(retries=5, backoff_factor=0.3, status_forcelist=(500, 502, 504), session=None):
    s = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist)
    adapter = HTTPAdapter(max_retries=retry)
    s.mount('http://', adapter)
    s.mount('https://', adapter)
    return s


def _call_public_api(url, **kwargs):
    try:
        resp = requests_retry_session().get(url, params=kwargs)
        contents = resp.json()
        return contents
    except Exception as x:
        print("It failed", x.__class__.__name__)
        return None
    else:
        print(resp.status_code)
        return None


def get_current_price(currency="BTC"):
    """
    최종 체결 가격을 얻어오는 메서드
    :param currency: BTC/BCH/BTG/ETH/ETC/XRP
    :return: price
    """
    try:
        currency = currency.lower() + "_krw"
        url = "https://api.korbit.co.kr/v1/ticker"
        contents = _call_public_api(url, currency_pair=currency)

        if contents is not None:
            price = contents['last']
            return float(price)
        else:
            return None
    except Exception as x:
        print(x.__class__.__name__)
        return None


def get_market_detail(currency="BTC"):
    """
    시장 현황 상세정보를 얻어오는 메서드
    :param currency: BTC/BCH/BTG/ETH/ETC/XRP
    :return: (24시간저가, 24시간고가, 최종체결가격, 거래량)
    """
    try:
        currency = currency.lower() + "_krw"
        url = "https://api.korbit.co.kr/v1/ticker/detailed"
        contents = _call_public_api(url, currency_pair=currency)

        if contents is not None:
            low  = contents['low']
            high = contents['high']
            last = contents['last']
            volume = contents['volume']
            return float(low), float(high), float(last), float(volume)
        else:
            return None, None, None, None
    except Exception as x:
        print(x.__class__.__name__)
        return None, None, None, None


def get_orderbook(currency="BTC"):
    """
    매수/매도 호가를 얻어오는 메서드
    :param currency: BTC/BCH/BTG/ETH/ETC/XRP
    :return: 매수/매도 호가
    """
    try:
        currency = currency.lower() + "_krw"
        url = "https://api.korbit.co.kr/v1/orderbook"
        contents = _call_public_api(url, currency_pair=currency)
        return contents
    except Exception as x:
        print(x.__class__.__name__)
        return None


def get_transaction_data(currency="BTC", interval="day"):
    """
    최근 1분/최근 1시간/최근 1일의 체결 데이터를 요청
    :param currency: BTC/BCH/BTG/ETH/ETC/XRP
    :param interval: minute/hour/day
    :return:
    """
    try:
        currency = currency.lower() + "_krw"
        url = "https://api.korbit.co.kr/v1/transactions"
        contents = _call_public_api(url, currency_pair=currency, time=interval)
        return contents
    except Exception as x:
        print(x.__class__.__name__)
        return None


def get_constants():
    """
    가상 화폐에 관련된 제약 조건을 얻어오는 메서드
    :return: Dict
    """
    try:
        url = "https://api.korbit.co.kr/v1/constants"
        contents = _call_public_api(url)
        if contents is not None:
            return contents['exchange']
        else:
            return None
    except Exception as x:
        print(x.__class__.__name__)
        return None


def get_tickers():
    try:
        constants = get_constants()
        return [x[:3].upper() for x in constants.keys()]
    except Exception as x:
        print(x.__class__.__name__)
        return None


if __name__ == "__main__":
    #----------------------------------------------------------------------------------------------
    # 최종 체결 가격
    #----------------------------------------------------------------------------------------------
    #print("BTC : ", get_current_price("BTC"))

    #----------------------------------------------------------------------------------------------
    # 시장 현황 상세정보
    #----------------------------------------------------------------------------------------------
    #print("BTC : ", get_market_detail("BTC"))

    #----------------------------------------------------------------------------------------------
    # 매수/매도 호가
    #----------------------------------------------------------------------------------------------
    #print("BTC : ", get_orderbook("BTC"))

    #----------------------------------------------------------------------------------------------
    # 체결 내역 (과거 데이터)
    #----------------------------------------------------------------------------------------------
    #print("BTC : ", get_transaction_data(currency="BTC", interval="day"))

    #----------------------------------------------------------------------------------------------
    # 제약조건
    #----------------------------------------------------------------------------------------------
    print(get_constants())

    print(get_tickers())

{'btc_krw': {'tick_size': 500.0, 'min_price': 1000, 'max_price': 100000000, 'order_min_size': 0.001, 'order_max_size': 100.0}, 'eth_krw': {'tick_size': 50.0, 'min_price': 1000, 'max_price': 100000000, 'order_min_size': 0.01, 'order_max_size': 1000.0}, 'etc_krw': {'tick_size': 10.0, 'min_price': 100, 'max_price': 100000000, 'order_min_size': 0.1, 'order_max_size': 20000.0}, 'xrp_krw': {'tick_size': 0.1, 'min_price': 10, 'max_price': 100000000, 'order_min_size': 10.0, 'order_max_size': 1000000.0}, 'bch_krw': {'tick_size': 50.0, 'min_price': 1000, 'max_price': 100000000, 'order_min_size': 0.005, 'order_max_size': 400.0}, 'btg_krw': {'tick_size': 500.0, 'min_price': 1000, 'max_price': 100000000, 'order_min_size': 0.005, 'order_max_size': 400.0}, 'ltc_krw': {'tick_size': 50.0, 'min_price': 1000, 'max_price': 100000000, 'order_min_size': 0.005, 'order_max_size': 500.0}, 'zil_krw': {'tick_size': 0.1, 'min_price': 1, 'max_price': 100000000, 'order_min_size': 1.0, 'order_max_size': 5000000.0}, 